In [ ]:
!pip install catboost
!pip install pygam


In [ ]:

import pandas as pd
import numpy as np
import warnings
from sklearn.linear_model import (HuberRegressor, RANSACRegressor, TheilSenRegressor,
                                  OrthogonalMatchingPursuit, PoissonRegressor,
                                  TweedieRegressor, RidgeCV, Lasso,
                                  ElasticNet, SGDRegressor, BayesianRidge)
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import r2_score
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor,
                              AdaBoostRegressor, BaggingRegressor, StackingRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from pygam import LinearGAM, s, f


warnings.simplefilter(action='ignore', category=Warning)

warnings.simplefilter(action='ignore', category=Warning)

# Load the dataset
data = pd.read_csv('path_to_your_file/file_name.csv')

# Log-transform the target variable
data['Cellular_Uptake_pg_Au_cell_log'] = np.log(data['Cellular_Uptake_pg_Au_cell'] + 1)

# Data preparation, you can change the column names to adjust your dataset
always_drop_columns = ['Row_number', 'row_number', 'Cell_source_system', 'Particle_ID', 'Coating_type', 'Coating_category', 'Coating category new','NP_mass_pg', 'Reference_DOI', 'Cellular_Uptake_pg_Au_cell_log', 'Cellular_uptake_number_of_NP', 'Cellular_Uptake_pg_Au_cell']
X = data.drop(columns=always_drop_columns + ['Coating_type', 'Coating_category', 'Coating category new'])
y = data['Cellular_Uptake_pg_Au_cell_log']

# Encoding categorical variables
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameters for tuning, use this part to ensure which hyperparameters are better for that individual regressor
"""rf_params = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

xgb_params = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.8, 0.9, 1]
}

bagging_params = {
    'n_estimators': [10, 20, 30],
    'max_samples': [0.5, 0.8, 1.0],
    'max_features': [0.5, 0.8, 1.0]
}

dt_params = {
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV for each model
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=5, n_jobs=-1)
rf_grid.fit(X_train, y_train)
print("Best parameters for Random Forest:", rf_grid.best_params_)

xgb_grid = GridSearchCV(XGBRegressor(random_state=42), xgb_params, cv=5, n_jobs=-1)
xgb_grid.fit(X_train, y_train)
print("Best parameters for XGBoost:", xgb_grid.best_params_)

bagging_grid = GridSearchCV(BaggingRegressor(random_state=42), bagging_params, cv=5, n_jobs=-1)
bagging_grid.fit(X_train, y_train)
print("Best parameters for Bagging:", bagging_grid.best_params_)

dt_grid = GridSearchCV(DecisionTreeRegressor(random_state=42), dt_params, cv=5, n_jobs=-1)
dt_grid.fit(X_train, y_train)
print("Best parameters for Decision Tree:", dt_grid.best_params_)"""


# Define and train various regressors with the training data, the hyperparameters in this code has been determined by running the codes between the """grid search""" statements

regressors = {
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_leaf=1, min_samples_split=2, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'Bagging': BaggingRegressor(n_estimators=30, max_features=1.0, max_samples=1.0, random_state=42),
    'Decision Tree': DecisionTreeRegressor(criterion='friedman_mse', max_depth=9, min_samples_leaf=2, min_samples_split=2, splitter='best', random_state=42),
    'Support Vector': SVR(),
    'K-Neighbors': KNeighborsRegressor(),
    'RidgeCV': RidgeCV(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'SGD': SGDRegressor(),
    'Gaussian Process': GaussianProcessRegressor(),
    'BayesianRidge': BayesianRidge(),
    'XGBoostRegressor': XGBRegressor(colsample_bytree=0.9, learning_rate=0.1, max_depth=7, n_estimators=200, subsample=0.8, random_state=42),
    'LightGBMRegressor': LGBMRegressor(),
    'SVR_poly': SVR(kernel='poly'),
    'SVR_sigmoid': SVR(kernel='sigmoid'),
    'SVR_rbf': SVR(kernel='rbf'),
    'HuberRegressor': HuberRegressor(),
    'RANSACRegressor': RANSACRegressor(),
    'TheilSenRegressor': TheilSenRegressor(),
    'KernelRidge': KernelRidge(),
    'OMP': OrthogonalMatchingPursuit(),
    'PoissonRegressor': PoissonRegressor(),
    'TweedieRegressor': TweedieRegressor()
}
results_log_transformed = []
for model_name, model in regressors.items():
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    results_log_transformed.append({
        'Model': model_name,
        'Train R2 Score': r2_train,
        'Test R2 Score': r2_test
    })

# Training and evaluating GAM separately
gam = LinearGAM(s(0) + s(1) + f(2)).fit(X_train, y_train)
gam_train_pred = gam.predict(X_train)
gam_test_pred = gam.predict(X_test)
results_log_transformed.append({
    'Model': 'GAM',
    'Train R2 Score': r2_score(y_train, gam_train_pred),
    'Test R2 Score': r2_score(y_test, gam_test_pred)
})
# Perform stacking using top models and append results to results_log_transformed,

stacked_models = []

for i in [2, 3, 4]:
    # Filter out already stacked models
    top_models = sorted([r for r in results_log_transformed if r['Model'] not in stacked_models],
                        key=lambda x: x['Test R2 Score'], reverse=True)[:i]

    estimators = [(model['Model'], regressors[model['Model']]) for model in top_models]

    stacking_regressor = StackingRegressor(estimators=estimators, cv=5)
    stacking_regressor.fit(X_train, y_train)
    stacked_train_pred = stacking_regressor.predict(X_train)
    stacked_test_pred = stacking_regressor.predict(X_test)

    stacked_r2_train = r2_score(y_train, stacked_train_pred)
    stacked_r2_test = r2_score(y_test, stacked_test_pred)

    stacked_model_name = f'StackingRegressor_Top{i}'
    results_log_transformed.append({
        'Model': stacked_model_name,
        'Train R2 Score': stacked_r2_train,
        'Test R2 Score': stacked_r2_test
    })
    stacked_models.append(stacked_model_name)

results_df = pd.DataFrame(results_log_transformed).sort_values(by='Test R2 Score', ascending=False)
print(results_df)

#by running this code snippet it is achiavable to get test and train R^2's of both individual and stacked models.


Since the R^2 values of the stacked models did not improved much, only individual regressors had been chosen to plot feature importances. The selection of algorithms for visualization was made by manually sorting the r^2 values. If you want to do this through code, you can modify your snippet with the line below.

**IMPORTANT NOTE:**  
Not every algorithm may contain the feature importance attribute.

In [ ]:
# Extract the names of the top-performing models
top_models = sorted(results_log_transformed, key=lambda x: x['Test R2 Score'], reverse=True)[:4]  # Top 4 models
top_model_names = [model['Model'] for model in top_models if model['Model'] in regressors and hasattr(regressors[model['Model']], 'feature_importances_')]

# Define positions of bars for each model
positions = np.array(range(len(sorted_idx)))

# Plotting bars
for idx, name in enumerate(top_model_names):
    values = importances[name]
    plt.barh(positions + width*idx, values[sorted_idx], color=colors[idx], label=name, height=width)


# **# FEATURE IMPORTANCE VISUALIZATION**

In [ ]:
plt.figure(figsize=(15, 12))

# Define the models you want to plot according to R^2's you got from running the code above
models_to_plot = ['XGBoostRegressor', 'Random Forest', 'Bagging', 'Gradient Boosting', 'LightGBMRegressor']

# Colors for the models
colors = ['#5081bc', '#7298bc', '#fe70bc', '#fab8dc', '#76b4a8']

# Normalize feature importances, the scale of the feature importances for LightGBMRegressor is different from others so normalizing would be a good option
for name, importance_values in importances.items():
    importances[name] = importance_values / np.sum(importance_values)

# Define positions of bars for each model
positions = np.array(range(len(sorted_idx)))

# Calculate width of a bar
width = 0.15

# Adjust the positions array to ensure bars are side-by-side
positions = positions - (width * len(models_to_plot) / 2)

# Plotting bars
for idx, name in enumerate(models_to_plot):
    values = importances[name]
    plt.barh(positions + width*idx, values[sorted_idx], color=colors[idx], label=name, height=width, align='center')

# Updating y-ticks to be in the center of grouped bars
plt.yticks(positions + width*(len(models_to_plot) - 1)/2, X.columns[sorted_idx])

plt.xlabel('Importance', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.title('Feature Importances\n', fontsize=16)
plt.legend(loc='best', fontsize=14)
plt.tight_layout()

# Directory to save the plot
output_directory = "path_for_saving_the_feature_importance_bar_graphs"

# Save the plot in PNG and TIFF formats, do not forget 1000 dpi for tiff could take large amount of space in your memory so adjust the resolution properly
plt.savefig(f"{output_directory}/Feature_Importances_results.png", format='png', dpi=1000)
plt.savefig(f"{output_directory}/Feature_Importances_results.tiff", format='tiff', dpi=1000)

plt.show()


# **RESIDUALS VISUALIZATION**

In [ ]:
import matplotlib.pyplot as plt

# Models for which you want to plot, you can add or remove any models that you want
model_names = ['XGBoostRegressor', 'Random Forest', 'Bagging', 'Gradient Boosting', 'LightGBMRegressor']

# Setting up the figure and axes
fig, axs = plt.subplots(nrows=len(model_names), ncols=1, figsize=(8, 6 * len(model_names)))

for ax, model_name in zip(axs, model_names):
    model = regressors[model_name]

    # Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Scatter plot without edgecolors
    ax.scatter(y_train, y_train_pred, alpha=0.6, s=20, label='Train', color='blue')
    ax.scatter(y_test, y_test_pred, alpha=0.6, s=20, label='Test', color='red')

    # Line for perfect prediction
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--k')

    # R^2 score for test and train data
    r2_test_value = r2_score(y_test, y_test_pred)
    r2_train_value = r2_score(y_train, y_train_pred)
    ax.text(0.05, 0.95, f'Test $R^2$: {r2_test_value:.2f}', fontsize=13, transform=ax.transAxes, verticalalignment='top')
    ax.text(0.05, 0.85, f'Train $R^2$: {r2_train_value:.2f}', fontsize=13, transform=ax.transAxes, verticalalignment='top')

    # Setting title, labels, and legend
    ax.set_title(f'Actual vs Predicted for {model_name}\n', fontsize=15)
    ax.set_xlabel('Log (Cellular uptake) - Actual', fontsize=14)
    ax.set_ylabel('Log (Cellular uptake) - Predicted', fontsize=14)
    ax.legend(loc='lower right', fontsize=12)

# Adjust layout
plt.tight_layout()
# Directory to save the plot
output_directory = "path_to_saving_directory"

# Save the plot in PNG and TIFF formats
plt.savefig(f"{output_directory}/Actual_vs_Predicted_results.png", format='png', dpi=1000)
plt.savefig(f"{output_directory}/Actual_vs_Predicted_results.tiff", format='tiff', dpi=1000)

plt.show()
